# Evaluación I Módulo 2 (sprint 2)
---

In [41]:
import pandas as pd
import numpy as np
import requests
import biblioteca as bb
from geopy.geocoders import Nominatim
import mysql.connector

En esta evaluación nos enfrentamos a un desafío emocionante de trabajar en un proyecto real para una empresa que realiza un estudio de universidades por el mundo. El proyecto tiene como objetivo identificar todas las universidades ubicadas en tres países específicos: Estados Unidos, Canadá y Argentina.  

Para llevar a cabo esta tarea, utilizaremos la API de "Universities Hipolabs", una fuente confiable y completa de información sobre las universidades en todo el mundo. Con la ayuda de esta API, podemos acceder a una gran cantidad de datos relevantes, incluyendo el nombre de la universidad, la ciudad donde esta ubicada, el nombre de la institución y otra información importante que nos permitirá llevar a cabo un análisis detallado.  

Es importante tener en cuenta que este proyecto requerirá un conocimiento profundo de herramientas y técnicas de análisis de datos, así como habilidades en programación y manejo de APIs. También es importante tener una comprensión sólida de la estructura y organización de los datos, ya que esto nos permitirá hacer preguntas importantes y obtener respuestas significativas a partir de los datos.  

En resumen, esta prueba técnica ofrece una excelente oportunidad para demostrar habilidades y conocimientos en análisis de datos y programación, mientras se trabaja en un proyecto real y relevante para una empresa. Al finalizar del proyecto, esperamos obtener información valiosa que ayudará a la empresa a tomar decisiones más informadas sobre las universidades en los tres países objetivo.  

1. Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es:
    ```python
    API_URL = "http://universities.hipolabs.com/search?country=NOMBREPAIS"
    ```

In [2]:
# los paises que nos interesan son:
paises = ['Argentina', 'Canada', 'United States']

In [3]:
# como vamos a tener que realizar la misma operación varias veces creamos una función para extraer la información:
def extraccion(pais):
    """Esta función realiza la extracción de los datos de la API de "Universities Hipolabs". Realiza la extracción de los 
    datos mostranndo tanto el código de estado como la razón del mismo.     
    Parámetros:
        - pais (str): país del que queremos extraer los datos
    Return: lista de diccionarios con los datos (formato .json).
    """
    url = f'http://universities.hipolabs.com/search?country={pais.lower()}'
    resp = requests.get(url=url)
    print(f'Status code extracción {pais}: {resp.status_code}')
    print(f'Reason extracción {pais}: {resp.reason}')
    return resp.json()

In [4]:
# obtenemos los datos de Argentina a modo de ejemplo para explorar los resultados que tendremos para cada país
argentina = extraccion(paises[0])
# el resultado obtenido es una lista de diccionarios
print(f'Nº de elementos de la lista resultado: {len(argentina)}') # obtenemos el número de elementos
print(f'Las keys de cada diccionario son {len(argentina[0])}: {argentina[0].keys()}')
print('Printeamos los dos primeros resultados descargados de la API a modo de ejemplo:')
argentina[0:2] # printeamos una muestra

Status code extracción Argentina: 200
Reason extracción Argentina: OK
Nº de elementos de la lista resultado: 87
Las keys de cada diccionario son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Printeamos los dos primeros resultados descargados de la API a modo de ejemplo:


[{'state-province': 'Buenos Aires',
  'domains': ['atlantida.edu.ar'],
  'name': 'Universidad Atlantida Argentina',
  'country': 'Argentina',
  'web_pages': ['http://www.atlantida.edu.ar/'],
  'alpha_two_code': 'AR'},
 {'state-province': 'Buenos Aires',
  'domains': ['austral.edu.ar'],
  'name': 'Universidad Austral Buenos Aires',
  'country': 'Argentina',
  'web_pages': ['http://www.austral.edu.ar/'],
  'alpha_two_code': 'AR'}]

In [5]:
# recordamos que teníamos la siguiente variable con los paises a analizar:
paises

['Argentina', 'Canada', 'United States']

In [6]:
# En este caso son solo tres paises pero por si fueran más podríamos automatizar la obtención de los datos, una vez conocida su estructura de la siguiente manera:
df = pd.DataFrame()
for pais in paises:
    datos = extraccion(pais)
    print(f'Nº de elementos de la lista resultado de {pais}: {len(datos)}') # obtenemos el número de elementos
    print(f'Las keys de cada diccionario de {pais} son {len(argentina[0])}: {argentina[0].keys()}')
    df_pais = pd.DataFrame(datos)
    print(f'Añadimos al dataframe resultado las {df.shape[0]} filas y {df.shape[1]} columnas de {pais}')
    df = pd.concat([df, df_pais], axis=0, ignore_index=True) # concatenamos los dataframes, ignorando el índice
    print('----------------------------------------')

Status code extracción Argentina: 200
Reason extracción Argentina: OK
Nº de elementos de la lista resultado de Argentina: 87
Las keys de cada diccionario de Argentina son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Añadimos al dataframe resultado las 0 filas y 0 columnas de Argentina
----------------------------------------
Status code extracción Canada: 200
Reason extracción Canada: OK
Nº de elementos de la lista resultado de Canada: 154
Las keys de cada diccionario de Canada son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages', 'alpha_two_code'])
Añadimos al dataframe resultado las 87 filas y 6 columnas de Canada
----------------------------------------
Status code extracción United States: 200
Reason extracción United States: OK
Nº de elementos de la lista resultado de United States: 2281
Las keys de cada diccionario de United States son 6: dict_keys(['state-province', 'domains', 'name', 'country', 'web_pages

In [7]:
# Antes de continuar realizamos una pequeña exploración del dataframe con la siguiente función
bb.explorar_df(df, nombre = 'Argentina-Canada-USA')

EXPLORACIÓN DEL DATAFRAME ARGENTINA-CANADA-USA
---------------------------------------------------------------------------
Las primeras 5 filas del dataframe Argentina-Canada-USA son:


,state-province,domains,name,country,web_pages,alpha_two_code
0,Buenos Aires,[atlantida.edu.ar],Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR
1,Buenos Aires,[austral.edu.ar],Universidad Austral Buenos Aires,Argentina,[http://www.austral.edu.ar/],AR
2,Ciudad Autónoma de Buenos Aires,[caece.edu.ar],"Universidad CAECE, Buenos Aires",Argentina,[http://www.caece.edu.ar/],AR
3,Ciudad Autónoma de Buenos Aires,[cema.edu.ar],Instituto Universitario CEMA,Argentina,[http://www.cema.edu.ar/],AR
4,Ciudad Autónoma de Buenos Aires,[iese.edu.ar],Instituto de Enseñanza Superior del Ejército,Argentina,[http://www.iese.edu.ar/],AR


---------------------------------------------------------------------------
Las últimas 5 filas del dataframe Argentina-Canada-USA son:


,state-province,domains,name,country,web_pages,alpha_two_code
2517,None,[vul.edu],Virginia University of Lynchburg,United States,[https://www.vul.edu/],US
2518,None,[voorhees.edu],Voorhees University,United States,[https://www.voorhees.edu/],US
2519,None,[wvstate.edu],West Virginia State University,United States,[https://www.wvstateu.edu/],US
2520,None,[wileyc.edu],Wiley College,United States,[https://www.wileyc.edu/],US
2521,None,[wssu.edu],Winston-Salem State University,United States,[https://www.wssu.edu/],US


---------------------------------------------------------------------------
A comntinuación se muestran 10 filas aleatorias del dataframe Argentina-Canada-USA:


...

---------------------------------------------------------------------------
El dataframe Argentina-Canada-USA tiene 2522 filas y 6 columnas
---------------------------------------------------------------------------
A continuación el resultado del método .info() incluyendo los tipos de dato de cada columna:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522 entries, 0 to 2521
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   state-province  273 non-null    object
 1   domains         2522 non-null   object
 2   name            2522 non-null   object
 3   country         2522 non-null   object
 4   web_pages       2522 non-null   object
 5   alpha_two_code  2522 non-null   object
dtypes: object(6)
memory usage: 118.3+ KB
---------------------------------------------------------------------------
El número de nulos por columna en valor absoluto y porcentaje es:
state-province: nº de nulos: 2249. % de nulos: 89

,count,unique,top,freq
state-province,273,49,Ontario,46
domains,2522,2491,[ccc.edu],7
name,2522,2507,Southeastern Community College,2
country,2522,3,United States,2281
web_pages,2522,2519,[http://www.coastalpines.edu/],2
alpha_two_code,2522,3,US,2281


---------------------------------------------------------------------------
Los principales estadísticos de las columnas categóricas son:


,count,unique,top,freq
state-province,273,49,Ontario,46
domains,2522,2491,[ccc.edu],7
name,2522,2507,Southeastern Community College,2
country,2522,3,United States,2281
web_pages,2522,2519,[http://www.coastalpines.edu/],2
alpha_two_code,2522,3,US,2281


---------------------------------------------------------------------------
El dataframe Argentina-Canada-USA tiene las siguientes columnas: 
Index(['state-province', 'domains', 'name', 'country', 'web_pages',
       'alpha_two_code'],
      dtype='object')
---------------------------------------------------------------------------
El numero de valores distintos de cada columna es:
state-province: 49
domains: 2491
name: 2507
country: 3
Los valores únicos de la columna 'country' son: ['Argentina' 'Canada' 'United States']
web_pages: 2519
alpha_two_code: 3
Los valores únicos de la columna 'alpha_two_code' son: ['AR' 'CA' 'US']


2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:  
    - Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.   
    - La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.  

In [8]:
# comprobamos que el índice es continuo
df.index

RangeIndex(start=0, stop=2522, step=1)

In [9]:
# antes de continuar realizamos una copia del dataframe original para realizar la limpieza sobre él
df2 = df.copy()
df2.sample()

,state-province,domains,name,country,web_pages,alpha_two_code
1550,None,[southark.edu],South Arkansas Community College,United States,[http://www.southark.edu],US


In [10]:
# ralizamos la homogeneización de columnas. Además de reemplazar los '-' por '_' utilizamos un .strip() por si hubiera algún espacio al inicio o el final que no vieramos
col_new = {col : col.strip().replace('-', '_') for col in df.columns} 
df2 = df2.rename(columns = col_new)
df2 = df2.reindex(columns=['country', 'state_province', 'web_pages', 'name', 'domains', 'alpha_two_code']) # ordenamos las columnas 
df2.sample()

,country,state_province,web_pages,name,domains,alpha_two_code
1506,United States,None,[http://www.lbwcc.edu],Lurleen B Wallace Community College,[lbwcc.edu],US


In [11]:
# eliminamos la columna con información suplicada
df2.drop(columns='domains', inplace=True)
df2.sample()

,country,state_province,web_pages,name,alpha_two_code
1156,United States,None,[http://www.tcu.edu/],Texas Christian University,US


3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva. [Aquí](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) tenéis la documentación de este método.

In [12]:
# comprobamos lo que nos indica el enunciado
for indice, row in df2.iterrows():
    if len(row['web_pages']) > 1:
        print(f'{row["name"]} ----> {row["web_pages"]}')


Cégep de Saint-Jérôme ----> ['https://www.cstj.qc.ca', 'https://ccmt.cstj.qc.ca', 'https://ccml.cstj.qc.ca']
Langara College ----> ['http://www.langara.bc.ca/', 'https://langara.ca/']
St. Mary's University ----> ['http://www.stmarys.ca/', 'https://smu.ca/']
Augusta University ----> ['http://www.augusta.edu/', 'http://www.gru.edu/']
California State University, Fresno ----> ['http://www.csufresno.edu/', 'http://www.fresnostate.edu/']
Capella University ----> ['http://www.capella.edu/', 'http://www.capellauniversity.edu/']
Colorado Technical University ----> ['http://www.coloradotech.edu/', 'http://www.ctuonline.edu/']
Thomas Edison State University ----> ['http://www.tesc.edu/', 'http://www.tesu.edu/']
United States Air Force Academy ----> ['http://www.usafa.af.mil/', 'http://www.usafa.edu/']
University of Texas Southwestern Medical Center ----> ['http://www.swmed.edu/', 'http://www.utsouthwestern.edu/']
University of the Incarnate Word ----> ['http://www.uiw.edu/', 'http://www.uiwtx.ed

In [13]:
# separamos con el método .explode()
df2 = df2.explode('web_pages')
print(f'El nuevo dataframe tiene {df2.shape[0]} filas y {df2.shape[1]} columnas')
df2.sample()

El nuevo dataframe tiene 2535 filas y 5 columnas


,country,state_province,web_pages,name,alpha_two_code
1353,United States,None,http://www.utsa.edu/,University of Texas at San Antonio,US


In [14]:
# nos fijamos en que tenemos un index no continuo
df2.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2512, 2513, 2514, 2515, 2516, 2517, 2518, 2519, 2520, 2521],
           dtype='int64', length=2535)

In [15]:
# reseteamos el índice
df2.reset_index(drop=True, inplace=True)
df2.sample()

,country,state_province,web_pages,name,alpha_two_code
777,United States,None,http://www.kings.edu/,King's College,US


In [16]:
# ahora sí es contínuo
df2.index

RangeIndex(start=0, stop=2535, step=1)

4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [17]:
# chequeamos que tenemos 28 duplicados si nos basamos en la columna 'name'
print(f'Si filtramos por la columna "name" tenemos {df2["name"].duplicated().sum()} duplicados')

Si filtramos por la columna "name" tenemos 28 duplicados


In [18]:
# antes de borrar los duplicados podríamos extraerlos por si nos interesase guardarlos y no perder los datos de las webs que vamos a eliminar del dataframe limpio (nos quedamos solo con la primera)
df_duplicados = df2[df2['name'].duplicated()]
print(f'El dataframe de los duplicados a eliminar tiene {df_duplicados.shape[0]} filas y {df_duplicados.shape[1]} columnas y se muestra a continuación:')
df_duplicados

El dataframe de los duplicados a eliminar tiene 28 filas y 5 columnas y se muestra a continuación:


,country,state_province,web_pages,name,alpha_two_code
88,Canada,Quebec,https://ccmt.cstj.qc.ca,Cégep de Saint-Jérôme,CA
89,Canada,Quebec,https://ccml.cstj.qc.ca,Cégep de Saint-Jérôme,CA
138,Canada,British Columbia,https://langara.ca/,Langara College,CA
179,Canada,None,https://smu.ca/,St. Mary's University,CA
410,United States,None,http://www.gru.edu/,Augusta University,US
478,United States,None,http://www.fresnostate.edu/,"California State University, Fresno",US
497,United States,None,http://www.capellauniversity.edu/,Capella University,US
563,United States,None,http://www.ctuonline.edu/,Colorado Technical University,US
601,United States,None,http://www.devry.edu/,DeVry Institute of Technology,US
777,United States,None,http://www.kings.edu/,King's College,US


In [19]:
# eliminamos los duplicados quedándonos con el primero en cada caso
df2.drop_duplicates(subset=['name'], inplace=True, ignore_index=True)
print(f'El nuevo dataframe tiene {df2.shape[0]} filas y {df2.shape[1]} columnas')
print('Comprobamos que el indice se ha reseteado:')
df2.index

El nuevo dataframe tiene 2507 filas y 5 columnas
Comprobamos que el indice se ha reseteado:


RangeIndex(start=0, stop=2507, step=1)

5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [20]:
# comprobamos que como dice el enunciado tenemos None
df2['state_province'].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', None,
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY'], dtype=object)

In [21]:
# utilizamos el método .fillna para sustituir None por np.nan
df2['state_province'].fillna(value=np.nan,  axis=None, inplace=True)
df2['state_province'].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', nan,
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY'], dtype=object)

6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [22]:
# aunque ya lo hemos visto en la exploración inicial recordamos
print(f'La columna "state_province" tiene un {round((df2["state_province"].isnull().sum() / df2.shape[0]) * 100, 2)} % de nulos')

La columna "state_province" tiene un 89.11 % de nulos


In [23]:
# los reemplazamos por 'Unknown'
df2['state_province'].replace(np.nan, 'Unknown', inplace=True)
df2['state_province'].value_counts()[:1] #comprobamos que se ha cambiado correctamente

Unknown    2234
Name: state_province, dtype: int64

7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso, [aquí](https://pypi.org/project/geopy/) la documentación. Para desarrollar este ejercicio deberéis:
    - Sacar los valores únicos de la columna state_province.
    - Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:
        - NV: reemplazalo por Nevada
        - TX: reemplazalo por Texas
        - IN: reemplazalo por Indianapolis
        - CA: reemplazalo por California
        - VA: reemplazalo por Virginia
        - NY: reemplazalo por New York
        - MI: reemplazalo por Michigan
        - GA: reemplazalo por Georgia
        - ND: reemplazalo por North Dakota
    - Otros valores que tenemos más formateados son y que deberemos reemplazar:
        - New York, NY. Deberéis reemplazarlo por "New York".
        - 'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires"
    - Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.
    - Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.
    - Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [24]:
# obtenemos los valores únicos
df2['state_province'].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Unknown',
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY'], dtype=object)

In [25]:
# para reemplazar todos los valores a la vez utilizamos .replace() con un diccionario
df2['state_province'] = df2['state_province'].replace({'NV': 'Nevada', 
                                                        'TX': 'Texas', 
                                                        'IN': 'Indianapolis',
                                                        'CA': 'California',
                                                        'VA': 'Virginia',
                                                        'NY': 'New York',
                                                        'MI': 'Michigan',
                                                        'GA': 'Georgia',
                                                        'ND': 'North Dakota',
                                                        'New York, NY' : 'New York',
                                                        'Ciudad Autónoma de Buenos Aires': 'Buenos Aires'})
df2['state_province'].unique()

array(['Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'Unknown', 'Santiago Del Estero', 'Misiones',
       'Catamarca', 'Formosa', 'Jujuy', 'La Rioja', 'La Pampa',
       'San Juan', 'San Luis', 'Tucumán', 'Quebec', 'Ontario',
       'Nova Scotia', 'British Columbia', 'Alberta', 'Manitoba',
       'New Brunswick', 'Saskatchewan', 'Newfoundland and Labrador',
       'Prince Edward Island', 'Yukon', 'Pennsylvania', 'Nevada', 'Iowa',
       'Virginia', 'Texas', 'Colorado', 'Indianapolis', 'California',
       'South Carolina', 'Washington', 'New York', 'North Dakota',
       'Michigan', 'Ohio', 'Florida', 'North Carolina', 'Georgia'],
      dtype=object)

In [26]:
# creamos la lista de las provincias con los valores únicos
provincias = df2['state_province'].unique().tolist()
provincias[:5] # printeamos una muestra como comprobación

['Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza']

In [27]:
print(f'Nº de provincias: {len(provincias)}')
print('Eliminamos el elemento "Unknown"')
provincias.remove('Unknown') # eliminamos este elemento para no solicitar sus coordenadas
print(f'Nº de provincias: {len(provincias)}')

Nº de provincias: 45
Eliminamos el elemento "Unknown"
Nº de provincias: 44


In [28]:
# usando la API de geopy obtenemos los datos
df_geopy = pd.DataFrame(columns=['state_province', 'latitude', 'longitude'])
for loc in provincias:
    geolocator = Nominatim(user_agent='Maitane') # inicializamos el geolocator
    location = geolocator.geocode(loc) # solicitamos la localización
    df = pd.DataFrame({'state_province': loc, 'latitude': location.latitude, 'longitude': location.longitude}, index=[0]) # obtenemos la latitud y longitud en un dataframe
    df_geopy = pd.concat([df_geopy, df], axis=0, ignore_index=True)
df_geopy.head()

,state_province,latitude,longitude
0,Buenos Aires,-34.607568,-58.437089
1,Entre Ríos,-31.625284,-59.353958
2,Salta,-25.107670,-64.349496
3,Córdoba,37.884581,-4.776014
4,Mendoza,-34.787093,-68.438187


In [30]:
# recordamos el dataframe que teníamos
print(f'El dataframe general tiene {df2.shape[0]} filas y {df2.shape[1]} columnas')
print('Comprobamos que el indice es contínuo:')
df2.index

El dataframe general tiene 2507 filas y 5 columnas
Comprobamos que el indice es contínuo:


RangeIndex(start=0, stop=2507, step=1)

In [31]:
# y el dataframe de las coordenadas tiene las sigueintes características
print(f'El dataframe de las coordenadas tiene {df_geopy.shape[0]} filas y {df_geopy.shape[1]} columnas')
print('Comprobamos que el indice es contínuo:')
df_geopy.index

El dataframe de las coordenadas tiene 44 filas y 3 columnas
Comprobamos que el indice es contínuo:


RangeIndex(start=0, stop=44, step=1)

In [32]:
# unimos ambos dataframes
df2 = df2.merge(df_geopy, how='left', on='state_province').reset_index(drop=True)
df2.head()

,country,state_province,web_pages,name,alpha_two_code,latitude,longitude
0,Argentina,Buenos Aires,http://www.atlantida.edu.ar/,Universidad Atlantida Argentina,AR,-34.607568,-58.437089
1,Argentina,Buenos Aires,http://www.austral.edu.ar/,Universidad Austral Buenos Aires,AR,-34.607568,-58.437089
2,Argentina,Buenos Aires,http://www.caece.edu.ar/,"Universidad CAECE, Buenos Aires",AR,-34.607568,-58.437089
3,Argentina,Buenos Aires,http://www.cema.edu.ar/,Instituto Universitario CEMA,AR,-34.607568,-58.437089
4,Argentina,Buenos Aires,http://www.iese.edu.ar/,Instituto de Enseñanza Superior del Ejército,AR,-34.607568,-58.437089


In [33]:
print(f'El dataframe completo tiene {df2.shape[0]} filas y {df2.shape[1]} columnas')
print('Comprobamos que el indice es contínuo:')
df2.index

El dataframe completo tiene 2507 filas y 7 columnas
Comprobamos que el indice es contínuo:


RangeIndex(start=0, stop=2507, step=1)

In [40]:
# comprobamos que en los casos en los que la 'state_province' es 'Unknown' tenemos np.nan en las columnas de latitid y longitud
df2[df2['state_province'] == 'Unknown'].sample(5)

,country,state_province,web_pages,name,alpha_two_code,latitude,longitude
814,United States,Unknown,http://www.manhattan.edu/,Manhattan College,US,NaN,NaN
1978,United States,Unknown,http://cdkc.edu,Chief Dull Knife College,US,NaN,NaN
2436,United States,Unknown,https://www.una.edu/,University of North Alabama,US,NaN,NaN
1944,United States,Unknown,http://www.sctcc.edu,St Cloud Technical and Community College,US,NaN,NaN
2140,United States,Unknown,http://www.belmontcollege.edu,Belmont College,US,NaN,NaN


8. Crea una BBDD en mysql que contenga las siguientes tablas:  

    ![imagen](../datos/8-bbdd.png)  
    
    - Tabla países: donde encontraremos las siguientes columnas:
        - idestado: primary key, integer, autoincremental
        - nombre_pais: varchar
        - nombre_provincia: varchar
        - latitud: decimal
        - longitud: decimal
    - Tabla universidades: donde encontraremos las siguientes columnas:
        - iduniversidades: primary key, integer, autoincremental
        - nombre_universidad: varchar
        - pagina_web: varchar
        - paises_idestado: foreing key

In [42]:
# creamos una función para crear la base de datos
def crear_bbdd(nombre_bbdd):
    
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab")
    
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [43]:
bbdd = crear_bbdd('bd_universidades')

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (111)

In [ ]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    
    mycursor = cnx.cursor()
    
    
    try: 
        mycursor.execute(query)
        cnx.commit() 
    
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [ ]:
tabla_univ = """CREATE TABLE IF NOT EXISTS `bd_universidades`.`paises` (
                    `idestado` INT NOT NULL AUTO_INCREMENT,
                    `nombre_pais` VARCHAR(45) NOT NULL,
                    `nombre_provincia` VARCHAR(45) NOT NULL,
                    `latitud` DECIMAL NOT NULL,
                    `longitud` DECIMAL NOT NULL,
                    PRIMARY KEY (`idestado`))
                    ENGINE = InnoDB;
                """

In [ ]:
tabla_paises = """ CREATE TABLE IF NOT EXISTS `bd_universidades`.`universidades` (
                        `iduniversidades` INT NOT NULL AUTO_INCREMENT,
                        `nombre_universidad` VARCHAR(100) NOT NULL,
                        `pagina_web` VARCHAR(100) NOT NULL,
                        `paises_idestado` INT NOT NULL,
                        PRIMARY KEY (`iduniversidades`),
                        INDEX `fk_universidades_paises_idx` (`paises_idestado` ASC) VISIBLE,
                        CONSTRAINT `fk_universidades_paises`
                            FOREIGN KEY (`paises_idestado`)
                            REFERENCES `bd_universidades`.`paises` (`idestado`)
                            ON DELETE CASCADE
                            ON UPDATE CASCADE)
                        ENGINE = InnoDB;
                    """

In [ ]:
crear_insertar_tabla('energia', 'AlumnaAdalab', tabla_univ)

In [ ]:
crear_insertar_tabla('energia', 'AlumnaAdalab', tabla_paises)

In [34]:
# -- MySQL Workbench Forward Engineering

# SET @OLD_UNIQUE_CHECKS=@@UNIQUE_CHECKS, UNIQUE_CHECKS=0;
# SET @OLD_FOREIGN_KEY_CHECKS=@@FOREIGN_KEY_CHECKS, FOREIGN_KEY_CHECKS=0;
# SET @OLD_SQL_MODE=@@SQL_MODE, SQL_MODE='ONLY_FULL_GROUP_BY,STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION';

# -- -----------------------------------------------------
# -- Schema bd_universidades
# -- -----------------------------------------------------

# -- -----------------------------------------------------
# -- Schema bd_universidades
# -- -----------------------------------------------------
# CREATE SCHEMA IF NOT EXISTS `bd_universidades` DEFAULT CHARACTER SET utf8 ;
# USE `bd_universidades` ;

# -- -----------------------------------------------------
# -- Table `bd_universidades`.`paises`
# -- -----------------------------------------------------



# -- -----------------------------------------------------
# -- Table `bd_universidades`.`universidades`
# -- -----------------------------------------------------



# SET SQL_MODE=@OLD_SQL_MODE;
# SET FOREIGN_KEY_CHECKS=@OLD_FOREIGN_KEY_CHECKS;
# SET UNIQUE_CHECKS=@OLD_UNIQUE_CHECKS;


9. Introduce todo el código que habéis ido creando en funciones, siguiendo la misma lógica que hemos seguido en los pairs

10. BONUS  
    - Introduce los datos en la BBDD de SQL.
    - Crea una clase con todo el código generado en esta evaluación.